# TP OBSPY

### To get started

We first start be importing the necesssary packages:

In [17]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install obspy folium

    100% |████████████████████████████████| 81kB 1.3MB/s ta 0:00:01
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
try:
    import obspy
    from obspy.core import read
    from obspy.clients.fdsn import Client
    from obspy import UTCDateTime
except ModuleNotFoundError:
    print("\nObspy package required!\n")
    raise

Absolute time values within ObsPy are handled with the **UTCDateTime** class. We define the *starttime* and the *endtime* of the signal:

In [8]:
starttime = UTCDateTime("2016-11-13T11:30:00")
endtime = UTCDateTime("2016-11-13T13:45:00")

We select the webservice FDSN RESIF client :

In [9]:
client_RESIF = Client("RESIF")

Here we will focus on the network __G__. To make an inventory of this network we use the **get_stations** function:

In [10]:
inventory = client_RESIF.get_stations(
    network="G",
    station="S*", # only station begining by S
    starttime=starttime,
    endtime=endtime,
)

In [11]:
print(inventory)

Inventory created at 2018-09-28T10:51:26.000000Z
	Created by: RESIF WEB SERVICE: http://ws.resif.fr/ws/station - version: 1.2
		    http://ws.resif.fr/fdsnws/station/1/query?starttime=2016-11-13T11%3...
	Sending institution: RESIF (RESIF)
	Contains:
		Networks (1):
			G
		Stations (3):
			G.SANVU (Espiritu Santo, Vanuatu)
			G.SPB (Sao Paulo, Brazil)
			G.SSB (Tunnel de Badole - Saint Sauveur en Rue, France)
		Channels (0):



### Select data and display some statistics

We choose the station **SSB** (Tunnel de Badole - Saint Sauveur en Rue, France).
We select the channel **BHZ** (high gain seismometer) and all the location (**\***). This is perform with the help of the **get_waveforms** function:

In [12]:
st = client_RESIF.get_waveforms(
    network="G",
    station="SSB",
    location="*",
    channel="BHZ",
    starttime=starttime,
    endtime=endtime,
)

We can display the general information about this stream. It contains three traces (data of a continuous series). It's a list, so the first trace is **st[0]**.

In [13]:
print(st)
first_trace = st[0] # or just st[0]...

3 Trace(s) in Stream:
G.SSB.00.BHZ | 2016-11-13T11:28:59.800000Z - 2016-11-13T13:46:13.000000Z | 20.0 Hz, 164665 samples
G.SSB.10.BHZ | 2016-11-13T11:29:42.000000Z - 2016-11-13T12:33:40.300000Z | 20.0 Hz, 76767 samples
G.SSB.10.BHZ | 2016-11-13T12:39:13.350000Z - 2016-11-13T13:45:56.900000Z | 20.0 Hz, 80072 samples


A Stats object may contain all header information of the trace:

In [14]:
stats = st[0].stats
print(stats)

               network: G
               station: SSB
              location: 00
               channel: BHZ
             starttime: 2016-11-13T11:28:59.800000Z
               endtime: 2016-11-13T13:46:13.000000Z
         sampling_rate: 20.0
                 delta: 0.05
                  npts: 164665
                 calib: 1.0
_fdsnws_dataselect_url: http://ws.resif.fr/fdsnws/dataselect/1/query
               _format: MSEED
                 mseed: AttribDict({'dataquality': 'Q', 'number_of_records': 79, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 4096, 'filesize': 598016})


### Where is it ?

We would like to display the station of interest on a map. To do this, we need to import additional modules.

In [15]:
import csv
import urllib.request

try:
    import folium
except ModuleNotFoundError:
    print("\nFolium package required!")
    raise


Folium package required!


ModuleNotFoundError: No module named 'folium'

We download the station information file provided by the RESIF web service:

In [ ]:
url = "http://ws.resif.fr/fdsnws/station/1/query?network=G&station=SSB&location=00&channel=BHZ&level=channel&format=text"
urllib.request.urlretrieve(url, "./station.txt")

Then we extract the latitude and the longitude of the station :

In [ ]:
with open("station.txt") as csvfile:
    reader = list(csv.reader(csvfile, delimiter="|"))
    # for row in reader:
    #    print(", ".join(row))

latitude = float(reader[1][4])
longitude = float(reader[1][5])
infos = " ".join(reader[1][0:4]) # network, station, location, channel

Then we add a marker on the map provided by the folium package (openstreetmap):

In [ ]:
background = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.Marker([latitude, longitude], tooltip=infos).add_to(background)
# background.save(outfile="station_map.html")

To display the map run the following cell:

In [ ]:
background

### Trace processing

Remove the mean:

In [ ]:
st[0].detrend("demean") # Keep in mind that this operation is made "in place". So st[0] is now modified.

Remove the trend:

In [ ]:
st[0].detrend("linear") # Remove the trend remove also the mean...

Apply a band-pass filter:

In [ ]:
st[0].filter(
    "bandpass",
    freqmin=0.001,
    freqmax=0.01,
)

Decimate (downsample) trace data by an integer factor:

In [ ]:
st[0].decimate(2)

Remove the instrumental response by deconvolution (This will return an error message.):

In [ ]:
try:
    st[0].remove_response()
except ValueError as ve:
    print(str(ve))

To perform this operation we need to import both the response function and the waveform (attach_response=True):

In [ ]:
# To perform the deconvolution we need to import both the response function and the waveform.
st = client_RESIF.get_waveforms(
    network="G",
    station="SSB",
    location="00",
    channel="BHZ",
    starttime=starttime,
    endtime=endtime,
    attach_response=True, # We attach the instrumental response.
)

We make a deep copy of the original trace and perform again some standard operation:

In [ ]:
raw_trace = st[0].copy() # Try without copy() function instead: raw_trace = st[0]
st[0].detrend("linear")
st[0].filter("bandpass", freqmin=0.001, freqmax=0.01)
st[0].decimate(2)
st[0].remove_response()

Visualisation of the original and process signals:

In [ ]:
raw_trace.plot()
st[0].plot()